In [1]:
!pip install pyarrow

In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00


In [3]:
# 필수 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
from sklearn.preprocessing import StandardScaler

import gc

In [4]:
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/Colab Notebooks/final_project/data/'

Mounted at /content/drive


In [5]:
# pyarrow 설치 (parquet 파일 읽기용)
!pip install pyarrow

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

In [6]:
# 데이터 로드: 6개월 신용정보 parquet 파일 병합
df21 = pd.read_parquet(data_root + 'test/2.신용정보/201807_test_신용정보.parquet')
df22 = pd.read_parquet(data_root + 'test/2.신용정보/201808_test_신용정보.parquet')
df23 = pd.read_parquet(data_root + 'test/2.신용정보/201809_test_신용정보.parquet')
df24 = pd.read_parquet(data_root + 'test/2.신용정보/201810_test_신용정보.parquet')
df25 = pd.read_parquet(data_root + 'test/2.신용정보/201811_test_신용정보.parquet')
df26 = pd.read_parquet(data_root + 'test/2.신용정보/201812_test_신용정보.parquet')

In [7]:
df2 = pd.concat([df21, df22, df23, df24, df25, df26])
print("병합 완료: ", df2.shape)

병합 완료:  (600000, 42)


In [8]:
# 불필요한 컬럼 제거
cols_to_drop = ['자발한도감액후경과월', '강제한도감액후경과월', '한도증액후경과월', '카드론동의여부', '한도심사요청후경과월', '한도심사거절후경과월']
df2.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print("컬럼 제거 완료")

컬럼 제거 완료


In [9]:
# 데이터 개요 확인
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600000 entries, 0 to 99999
Data columns (total 36 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   기준년월            600000 non-null  int64  
 1   ID              600000 non-null  object 
 2   최초한도금액          600000 non-null  int64  
 3   카드이용한도금액        600000 non-null  int64  
 4   CA한도금액          600000 non-null  int64  
 5   일시상환론한도금액       600000 non-null  int64  
 6   월상환론한도금액        600000 non-null  int64  
 7   CA이자율_할인전       600000 non-null  float64
 8   CL이자율_할인전       600000 non-null  float64
 9   RV일시불이자율_할인전    600000 non-null  float64
 10  RV현금서비스이자율_할인전  600000 non-null  float64
 11  RV신청일자          112482 non-null  float64
 12  RV약정청구율         600000 non-null  float64
 13  RV최소결제비율        600000 non-null  float64
 14  자발한도감액횟수_R12M   600000 non-null  object 
 15  자발한도감액금액_R12M   600000 non-null  int64  
 16  강제한도감액횟수_R12M   600000 non-null  int64  
 17  강제한도감액금액_R12M   

In [10]:
# 결측치 비율 너무 높아서(81%) RV신청일자 컬럼 삭제
cols_to_drop = ['RV신청일자']
df2.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print("컬럼 제거 완료")

컬럼 제거 완료


In [11]:
# RV전환가능여부 결측치를 최빈값 'Z'로 대체
df2['RV전환가능여부'] = df2['RV전환가능여부'].fillna('Z')

In [12]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600000 entries, 0 to 99999
Data columns (total 35 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   기준년월            600000 non-null  int64  
 1   ID              600000 non-null  object 
 2   최초한도금액          600000 non-null  int64  
 3   카드이용한도금액        600000 non-null  int64  
 4   CA한도금액          600000 non-null  int64  
 5   일시상환론한도금액       600000 non-null  int64  
 6   월상환론한도금액        600000 non-null  int64  
 7   CA이자율_할인전       600000 non-null  float64
 8   CL이자율_할인전       600000 non-null  float64
 9   RV일시불이자율_할인전    600000 non-null  float64
 10  RV현금서비스이자율_할인전  600000 non-null  float64
 11  RV약정청구율         600000 non-null  float64
 12  RV최소결제비율        600000 non-null  float64
 13  자발한도감액횟수_R12M   600000 non-null  object 
 14  자발한도감액금액_R12M   600000 non-null  int64  
 15  강제한도감액횟수_R12M   600000 non-null  int64  
 16  강제한도감액금액_R12M   600000 non-null  int64  
 17  한도증액횟수_R12M     

In [13]:
# 결측치 재확인
missing_cols = df2.columns[df2.isnull().any()]
missing_cols

Index([], dtype='object')

vif 기준 피처 제거

In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# 수치형 컬럼만 선택
numeric_cols = df2.select_dtypes(include=['float64', 'int64']).columns.tolist()
X_vif = df2[numeric_cols].dropna()

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# VIF가 10 이상인 피처 확인
high_vif_features = vif_data[vif_data["VIF"] > 10]["Feature"].tolist()
print("VIF > 10:", high_vif_features)

VIF > 10: ['기준년월', '카드이용한도금액', 'RV일시불이자율_할인전', 'RV약정청구율', '상향가능한도금액', '상향가능CA한도금액', '카드이용한도금액_B1M', '카드이용한도금액_B2M', 'rv최초시작후경과일']


In [16]:
# 1. 기준년월 컬럼을 제외한 VIF 10 이상 컬럼만 제거 대상에 포함
features_to_drop = [col for col in high_vif_features if col != '기준년월']

# 2. 컬럼 제거
df2 = df2.drop(columns=features_to_drop)
print("제거된 컬럼 목록:", features_to_drop)

제거된 컬럼 목록: ['카드이용한도금액', 'RV일시불이자율_할인전', 'RV약정청구율', '상향가능한도금액', '상향가능CA한도금액', '카드이용한도금액_B1M', '카드이용한도금액_B2M', 'rv최초시작후경과일']


In [17]:
# 기준년월이 여전히 남아있는지 확인
print("'기준년월' in df2.columns:", '기준년월' in df2.columns)

'기준년월' in df2.columns: True


In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   기준년월            600000 non-null  int64  
 1   ID              600000 non-null  object 
 2   최초한도금액          600000 non-null  int64  
 3   CA한도금액          600000 non-null  int64  
 4   일시상환론한도금액       600000 non-null  int64  
 5   월상환론한도금액        600000 non-null  int64  
 6   CA이자율_할인전       600000 non-null  float64
 7   CL이자율_할인전       600000 non-null  float64
 8   RV현금서비스이자율_할인전  600000 non-null  float64
 9   RV최소결제비율        600000 non-null  float64
 10  자발한도감액횟수_R12M   600000 non-null  object 
 11  자발한도감액금액_R12M   600000 non-null  int64  
 12  강제한도감액횟수_R12M   600000 non-null  int64  
 13  강제한도감액금액_R12M   600000 non-null  int64  
 14  한도증액횟수_R12M     600000 non-null  object 
 15  한도증액금액_R12M     600000 non-null  int64  
 16  월상환론상향가능한도금액    600000 non-null  int64  
 17  RV전환가능여부        

In [19]:
# Parquet 저장(세그먼트 포함)
df2.to_parquet("test_2번eda.parquet", index=False)

# 파일 다운로드
from google.colab import files
files.download("test_2번eda.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

인코딩

In [20]:
from sklearn.preprocessing import LabelEncoder

# 범주형 컬럼 확인 및 인코딩
categorical_cols = df2.select_dtypes(include=['object', 'category']).columns

# Label Encoding 수행
for col in categorical_cols:
    le = LabelEncoder()
    df2[col] = le.fit_transform(df2[col].astype(str)) # 결측치 등을 문자열 'nan'으로 처리하여 인코딩

print("범주형 컬럼 인코딩 완료")

# Parquet 저장
df2.to_parquet("test_2번eda_encoded.parquet", index=False)

# 파일 다운로드
from google.colab import files
files.download("test_2번eda_encoded.parquet")

범주형 컬럼 인코딩 완료


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>